In [4]:
import sys,os

#import locale
from datetime import datetime

#locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

current_directory = os.getcwd()
sys.path.insert(0,f'{current_directory}/../src/')
sys.path.insert(0,f'{current_directory}/../chromedriver-mac-x64/chromedriver')

from playtomic_tools import get_playtomic_schedule,add_playtomic_schedule

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.caches import BaseCache
from langchain_core.callbacks import Callbacks
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory


os.environ["OPENAI_API_KEY"] = str(os.getenv("OPENAI_API_KEY"))

In [14]:
openai_model = "gpt-4o-mini"
# The extra thing
ChatOpenAI.model_rebuild()

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [ ]:
tools = [get_playtomic_schedule]

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=False
)

In [ ]:
out = agent_executor.invoke({
    "input": f"",
    "chat_history": memory.chat_memory.messages,
})